In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
basefolder = 'C:/Users/karim/Documents/westgate/data/training/'

In [3]:
import pandas as pd
from westgate.flaml_model_trainer import UWModelTrainer
from westgate.flaml_model_core import UWModel
from westgate.flaml_model_utils import load_model
import pandas as pd
from westgate.combochart import combo_chart
import logging
import numpy as np

In [4]:
logger = logging.getLogger('westgate.flaml_model')
fhandler = logging.FileHandler(filename='metrics.log', mode='w')
formatter = logging.Formatter('%(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

In [5]:
default_model = UWModel()

default_model_trainer = UWModelTrainer(model_name='default', model_core=default_model, model_version='1.0')

In [6]:
data_2022_df = pd.read_csv(basefolder + 'loan_outcomes_2022.csv', encoding='latin')

C:\Users\karim\AppData\Local\Temp\ipykernel_26596\2959394401.py:1: DtypeWarning: Columns (741) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2022_df = pd.read_csv(basefolder + 'loan_outcomes_2022.csv', encoding='latin')


In [7]:
jan2023_df = pd.read_csv(basefolder + 'loan_outcomes_jan2023.csv', encoding='latin')
feb2023_df = pd.read_csv(basefolder + 'loan_outcomes_feb2023.csv', encoding='latin')
mar2023_df = pd.read_csv(basefolder + 'loan_outcomes_mar2023.csv', encoding='latin')
apr2023_df = pd.read_csv(basefolder + 'loan_outcomes_apr2023.csv', encoding='latin')
may2023_df = pd.read_csv(basefolder + 'loan_outcomes_may2023.csv', encoding='latin')

In [8]:
accepted_df = pd.concat([data_2022_df, jan2023_df, feb2023_df, mar2023_df, apr2023_df, may2023_df])

In [9]:
len(accepted_df)

14680

In [10]:
accepted_df['request_yyyymm'] = accepted_df['request_date'].str[0:7]
accepted_df['validation_flag'] = accepted_df['request_yyyymm'].isin(['2023-01', '2023-02'])
accepted_df['test_flag'] = accepted_df['request_yyyymm'].isin(['2023-03'])
accepted_df['weight'] = np.select(
    [
        accepted_df['request_yyyymm'] == '2022-01',
        accepted_df['request_yyyymm'] == '2022-02',
        accepted_df['request_yyyymm'] == '2022-03',
        accepted_df['request_yyyymm'] == '2022-04',
        accepted_df['request_yyyymm'] == '2022-05',
        accepted_df['request_yyyymm'] == '2022-06',
        accepted_df['request_yyyymm'] == '2022-07',
        accepted_df['request_yyyymm'] == '2022-08',
        accepted_df['request_yyyymm'] == '2022-09',
        accepted_df['request_yyyymm'] == '2022-10',
        accepted_df['request_yyyymm'] == '2022-11',
        accepted_df['request_yyyymm'] == '2022-12',
        accepted_df['request_yyyymm'] == '2023-01',
        accepted_df['request_yyyymm'] == '2023-02',
        accepted_df['request_yyyymm'] == '2023-03',
        accepted_df['request_yyyymm'] == '2023-04',
        accepted_df['request_yyyymm'] == '2023-05'
    ],
    [
        1/12,
        2/12,
        3/12,
        4/12,
        5/12,
        6/12,
        7/12,
        8/12,
        9/12,
        10/12,
        11/12,
        1,
        1,
        1,
        1,
        1,
        1
    ],
    default=1
)


In [11]:
accepted_df['profit'] = accepted_df['total_paid'] - accepted_df['principal']

In [12]:
accepted_df[default_model.target] = np.where(accepted_df['profit'] < 50, 1, 0)

In [13]:
accepted_df[default_model.target].value_counts(normalize=True)

status
0    0.711172
1    0.288828
Name: proportion, dtype: float64

In [14]:
filtered_df = default_model.filter_df(accepted_df)

189 rows removed by threshold filtering
458 Rows with 'error' column not NA have been discarded.
5 Rows with 'Id' column NA have been discarded.
36 Rows with 'account_age_days' column not positive have been discarded.


In [15]:
len(filtered_df)

13992

In [16]:
filtered_df[default_model.target].value_counts(normalize=True)

status
0    0.712193
1    0.287807
Name: proportion, dtype: float64

In [17]:
# here X_teste is December

X_train, X_test, y_train, y_test, extra = \
    default_model_trainer.split_data(filtered_df, 
                                    split_criteria='test_flag',
                                    extra_cols=['weight', 'validation_flag', 'request_yyyymm'])

X_train, X_test = default_model_trainer.feature_engineer(X_train, X_test)

#TODO: deal with na values in X_train/X_test if any
#non_na_idx = X['age'].notna()

#X = X[non_na_idx]

#if y is not None:
    #y = y[non_na_idx]

In [18]:
extra.keys()

dict_keys(['train_weight', 'test_weight', 'train_validation_flag', 'test_validation_flag', 'train_request_yyyymm', 'test_request_yyyymm', 'train_profit', 'test_profit'])

In [19]:
len(X_train)

13152

In [20]:
X_val, y_val = X_train.loc[extra['train_validation_flag']], y_train.loc[extra['train_validation_flag']]
X_train, y_train = X_train.loc[~extra['train_validation_flag']], y_train.loc[~extra['train_validation_flag']]

In [21]:
len(X_train) + len(X_val) + len(X_test) == len(filtered_df)

True

In [22]:
weight_train = extra['train_weight'].loc[~extra['train_validation_flag']]
weight_val = extra['train_weight'].loc[extra['train_validation_flag']]
weight_test = extra['test_weight']

In [23]:
len(weight_train), len(weight_val), len(weight_test)

(11267, 1885, 840)

In [24]:
len(X_train), len(X_val), len(X_test)

(11267, 1885, 840)

In [25]:
def p90_accuracy_metric(X_val, y_val, estimator, labels,
                X_train, y_train, weight_val=None, weight_train=None,
                *args):
    y_probas = estimator.predict_proba(X_val)[:, 1]
    df = pd.DataFrame({'proba': y_probas, 'y_val': y_val})
    perc90 = np.percentile(y_probas, 90)
    perc10 = np.percentile(y_probas, 10)
    delta = df.loc[df['proba'] >= perc90]['y_val'].sum() - df.loc[df['proba'] <= perc10]['y_val'].sum()
    return -delta, {}
        
config = {
    #"ensemble": True,
    "estimator_list": ['xgb_limitdepth'],
    #"estimator_list": ['xgboost'],
    "metric": "roc_auc",
    #"metric": p90_accuracy_metric,
    "eval_method": 'holdout',
    'X_val': X_val,
    'y_val': y_val,
    'sample_weight': weight_train,
    'sample_weight_val': weight_val
}

y_pred_proba, y_pred, extra = default_model_trainer.fit(
        X_train, X_test, y_train, y_test, extra,
        time_budget=150, 
        automl_config=config,
        show_plots=True,
        percentile=90
)

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [43]:
len(default_model.features)

34

In [44]:
print(default_model.features)

['average_closing_balance_day_after_employer_income', 'average_closing_balance_day_of_employer_income', 'average_closing_balance_day_of_income', 'average_monthly_micro_loan_payments_complex', 'average_monthly_utility_payments_complex', 'balance_90_days_ago', 'balance_current', 'balance_min', 'count_days_negative_balance_90_days', 'count_distinct_micro_lenders', 'count_loan_payments_current_month', 'count_loan_payments_previous_month', 'count_nsf_30_days', 'employer_income_frequency', 'sum_employer_income_2_months_ago', 'sum_employer_income_current_month', 'sum_employer_income_previous_month', 'sum_government_income_2_months_ago', 'sum_government_income_current_month', 'sum_government_income_previous_month', 'sum_loan_deposits_90_days', 'sum_non_employer_income', 'sum_non_employer_income_2_months_ago', 'sum_non_employer_income_current_month', 'sum_non_employer_income_previous_month', 'telecom_payments_average', 'other_loan_payments_average', 'utility_payments_average', 'gender', 'age', 

In [45]:
Xfull = pd.concat([X_train, X_val, X_test])

In [46]:
yfull = pd.concat([y_train, y_val, y_test])

In [47]:
weight_full = pd.concat([weight_train, weight_val, weight_test])

In [50]:
default_model_trainer.retrain_full(
    Xfull,
    yfull,
    weight_full=weight_full,
    time_budget=60
)

In [34]:
default_model.percentiles = {p:v for p,v in zip(range(5,100,5), percentiles)}

Saving / loading the model

In [35]:
default_model.save()

In [36]:
default_model = load_model('default')

In [37]:
default_model.feat_imp()

,variable,imp
13,count_loan_payments_previous_month,0.138593
5,average_monthly_micro_loan_payments_complex,0.129439
11,count_distinct_micro_lenders,0.064167
23,other_loan_payments_average,0.039477
8,balance_current,0.035601
17,sum_employer_income_previous_month,0.035590
6,average_monthly_utility_payments_complex,0.034482
15,sum_employer_income_2_months_ago,0.031936
0,employer_income_frequency,0.031836
4,average_closing_balance_day_of_income,0.031080


In [27]:
default_model.save()

# Predictions for Jan'23

In [42]:
data2023_df = pd.read_csv('../data/janToNov2023Loansv2.csv', encoding='latin')

C:\Users\karim\AppData\Local\Temp\ipykernel_3504\4126847242.py:1: DtypeWarning: Columns (387) have mixed types. Specify dtype option on import or set low_memory=False.
  data2023_df = pd.read_csv('../data/janToNov2023Loansv2.csv', encoding='latin')


In [43]:
len(data2023_df)

9499

In [44]:
jan23_df = data2023_df.loc[data2023_df['request_date'].str[:7]=='2023-01']

In [45]:
len(jan23_df)

1045

In [46]:
jan23_df.drop_duplicates(inplace=True)

In [47]:
len(jan23_df)

1040

In [58]:
jan23_df.set_index('loanId', inplace=True)

In [59]:
jan23_df.head()

,dob,gender,request_date,account_age_days,active_days_trend,auto_loan_payment_frequency,average_closing_balance_day_after_employer_income,average_closing_balance_day_after_government_income,average_closing_balance_day_after_income,average_closing_balance_day_of_employer_income,...,sum_wsib_income,student_loan_payments_frequency,telecom_payments_frequency,telecom_payments_average,other_loan_payments_frequency,other_loan_payments_average,utility_payments_frequency,utility_payments_average,average_monthly_insurance_income_complex,MostRecentTransactionDate
loanId,,,,,,,,,,,,,,,,,,,,,
3970620,1991-09-15,Female,2023-01-19,366.0,1.03,Monthly,364.90,85.89,297.83,329.04,...,0.0,Unknown,Unknown,143.38,BiWeekly,293.26,NONE_DETECTED,0.00,7.27,05/25/2023
3986502,1991-11-07,Female,2023-01-31,365.0,1.02,NONE_DETECTED,381.74,269.92,276.69,411.25,...,0.0,NONE_DETECTED,Unknown,130.27,NONE_DETECTED,0.00,NONE_DETECTED,0.00,0.00,06/19/2023
3813879,1989-08-16,Male,2023-01-02,366.0,1.06,Weekly,996.93,-55.76,835.17,1153.10,...,0.0,NONE_DETECTED,NONE_DETECTED,0.00,Weekly,83.37,NONE_DETECTED,0.00,0.00,08/29/2023
3814985,2004-02-04,Male,2023-01-03,365.0,0.74,NONE_DETECTED,1178.11,1.26,1114.36,1249.69,...,0.0,NONE_DETECTED,Monthly,40.40,NONE_DETECTED,0.00,Unknown,76.93,0.40,04/21/2023
3813995,1992-08-21,Male,2023-01-02,366.0,1.08,NONE_DETECTED,522.96,170.62,429.96,525.33,...,0.0,NONE_DETECTED,Unknown,163.80,BiWeekly,279.30,Unknown,157.44,1.08,08/07/2023


In [48]:
loans23_df = pd.read_csv('../data/loans_branch-if-jan2023.csv')

In [49]:
loans23_df['profit'] = loans23_df['Paid Amount'] - loans23_df['Principal Amount']

In [50]:
loans23_df.set_index('Loan Id', inplace=True)

In [55]:
len(loans23_df)

31510

In [56]:
loans23_df.sort_index().head()

,Loan #,Loan Duration,Maturity Date,First Name,Last Name,Full Name,Released Date,Principal Amount,Disbursed By,Paid Amount,...,Total Interest Paid,Total Penalty Balance,Total Penalty Paid,Pending Due,Pending Fees Due,Pending Interest Due,Pending Penalty Due,Pending Principal Due,Balance Amount,profit
Loan Id,,,,,,,,,,,,,,,,,,,,,
2545807,IF10000083,3 Months,02/24/2022,Brian,Robert Wason,Brian Robert Wason,12/02/2021,300.0,Cash,587.58,...,17.58,0.0,0.0,0.00,NaN,NaN,NaN,NaN,0.00,287.58
2545954,IF10000085,3 Months,02/18/2022,Ashley,Sarah Pulker,Ashley Sarah Pulker,12/02/2021,300.0,Online Transfer,391.72,...,11.72,110.0,0.0,305.86,90.0,5.86,110.0,100.0,305.86,91.72
2545977,IF10000086,2 Months,01/31/2022,Anpalagan,Dilajini,Anpalagan Dilajini,12/02/2021,200.0,Cash,379.74,...,7.74,0.0,0.0,0.00,90.0,5.86,110.0,100.0,0.00,179.74
2545990,IF10000087,3 Months,02/24/2022,Kenneth,Payne,Kenneth Payne,12/02/2021,350.0,Cash,880.51,...,20.51,0.0,210.0,0.00,90.0,5.86,110.0,100.0,0.00,530.51
2546124,IF10000091,3 Months,02/24/2022,Dawn Eleanor,Bailey,Dawn Eleanor Bailey,12/02/2021,300.0,Cash,587.58,...,17.58,0.0,0.0,0.00,90.0,5.86,110.0,100.0,0.00,287.58


In [60]:
jan23_ex_df = pd.merge(jan23_df, loans23_df, left_index=True, right_index=True, validate='one_to_one')

In [61]:
len(jan23_ex_df)

1040

In [62]:
jan23_ex_df['pred_proba'] = default_model.predict_proba(jan23_ex_df)['pred_proba']

In [63]:
predicted_defaults_df = jan23_ex_df.loc[jan23_ex_df['pred_proba'] >= p90_2022]

In [64]:
len(predicted_defaults_df), len(jan23_ex_df), len(predicted_defaults_df) / len(jan23_ex_df) 

(74, 1040, 0.07115384615384615)

In [65]:
profit_jan23 = predicted_defaults_df['profit'].sum() / len(predicted_defaults_df)
profit_jan23

-16.31202702702702

# Predictions for all 2022 + jan'23

In [66]:
request_month = pd.concat([extra['train_request_month'], extra['test_request_month']])
profits = pd.concat([extra['train_profit'], extra['test_profit']])

In [67]:
res_list = []

for month in range(1, 13):
    X_month = X2022.loc[request_month == month]
    profits_month = profits.loc[request_month == month]
    preds_df = default_model.predict_proba(X_month, filter=False, engineer=False)
    preds_df['profit'] = profits_month
    predicted_defaults = preds_df.loc[preds_df['pred_proba'] >= p90_2022]
    res_list.append({'month': '2022-' + (str(month) if month >= 10 else '0' + str(month)), 
                    'anti-profit per loan': round(predicted_defaults['profit'].sum() / len(predicted_defaults))})

profits_month = jan23_ex_df['profit']
preds_df = default_model.predict_proba(jan23_ex_df)
preds_df['profit'] = profits_month
predicted_defaults = preds_df.loc[preds_df['pred_proba'] >= p90_2022]
res_list.append({'month': '2023-01', 
                'anti-profit per loan': round(predicted_defaults['profit'].sum() / len(predicted_defaults))})

In [68]:
pd.DataFrame.from_records(res_list)

,month,anti-profit per loan
0,2022-01,-89
1,2022-02,-33
2,2022-03,42
3,2022-04,36
4,2022-05,53
5,2022-06,42
6,2022-07,13
7,2022-08,12
8,2022-09,30
9,2022-10,10


In [236]:
pd.DataFrame.from_records(res_list)['anti-profit per loan'].mean()

33.38461538461539

In [39]:
for handler in logger.handlers:
    logger.removeHandler(handler)
    handler.close()

## Debugging

In [41]:
default_model = load_model('default')

In [42]:
default_model.features_in

['request_date',
 'dob',
 'employer_income_frequency',
 'gender',
 'average_closing_balance_day_after_employer_income',
 'average_closing_balance_day_of_employer_income',
 'balance_90_days_ago',
 'balance_current',
 'balance_min',
 'count_loan_payments_current_month',
 'count_loan_payments_previous_month',
 'count_nsf_30_days',
 'sum_employer_income_current_month',
 'sum_employer_income_previous_month',
 'sum_employer_income_2_months_ago',
 'count_distinct_micro_lenders',
 'count_days_negative_balance_90_days',
 'sum_non_employer_income_current_month',
 'sum_non_employer_income_previous_month',
 'sum_non_employer_income_2_months_ago',
 'sum_non_employer_income',
 'average_closing_balance_day_of_income',
 'average_monthly_micro_loan_payments_complex',
 'average_monthly_utility_payments_complex',
 'telecom_payments_average',
 'other_loan_payments_average',
 'utility_payments_average']